In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import seaborn as sns

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.multiclass import OneVsRestClassifier

In [11]:
raw_data_path = os.path.join(os.path.pardir,'data','raw')
dataset_file_path = os.path.join(raw_data_path,'stackoverflow.csv')
dataset_file_path

'..\\data\\raw\\stackoverflow.csv'

In [12]:
df = pd.read_csv(dataset_file_path)
df.shape

(48976, 3)

In [8]:
df.head()

,Unnamed: 0,Text,Tags
0,2,aspnet site maps has anyone got experience cre...,"['sql', 'asp.net']"
1,4,adding scripting functionality to net applicat...,"['c#', '.net']"
2,5,should i use nested classes in this case i am ...,['c++']
3,6,homegrown consumption of web services i have b...,['.net']
4,8,automatically update version number i would li...,['c#']


In [14]:
df['Tags'].iloc[0]

"['sql', 'asp.net']"

In [15]:
type(df['Tags'].iloc[0])

str

In [16]:
import ast
ast.literal_eval(df['Tags'].iloc[0])

['sql', 'asp.net']

In [17]:
df['Tags'] = df['Tags'].apply(lambda x: ast.literal_eval(x))

In [18]:
df.head()

,Unnamed: 0,Text,Tags
0,2,aspnet site maps has anyone got experience cre...,"[sql, asp.net]"
1,4,adding scripting functionality to net applicat...,"[c#, .net]"
2,5,should i use nested classes in this case i am ...,[c++]
3,6,homegrown consumption of web services i have b...,[.net]
4,8,automatically update version number i would li...,[c#]


In [20]:
y = df['Tags']
y

0        [sql, asp.net]
1            [c#, .net]
2                 [c++]
3                [.net]
4                  [c#]
              ...      
48971             [c++]
48972             [c++]
48973          [python]
48974          [python]
48975             [c++]
Name: Tags, Length: 48976, dtype: object

In [21]:
multilabel = MultiLabelBinarizer()

In [22]:
y = multilabel.fit_transform(df['Tags'])

In [23]:
y

array([[0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
multilabel.classes_

array(['.net', 'android', 'asp.net', 'c', 'c#', 'c++', 'css', 'html',
       'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby', 'ruby-on-rails', 'sql'],
      dtype=object)

In [25]:
pd.DataFrame(y,columns=multilabel.classes_)

,.net,android,asp.net,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,mysql,objective-c,php,python,ruby,ruby-on-rails,sql
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48971,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48972,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48973,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [26]:
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,3), stop_words='english')
X = tfidf.fit_transform(df['Text'])

In [27]:
X

<48976x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 2002390 stored elements in Compressed Sparse Row format>

In [28]:
X.shape, y.shape

((48976, 10000), (48976, 20))

In [29]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=0)

# Build Model

In [33]:
sgd = SGDClassifier()
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
svc = LinearSVC()

In [31]:
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
  return jaccard.mean()*100


def print_score(y_pred, clf):
  print("Clf: ", clf.__class__.__name__)
  print('Jacard score: {}'.format(j_score(y_test, y_pred)))
  print('----')

In [34]:
for classifier in [sgd, lr, svc]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print_score(y_pred, classifier)

Clf:  SGDClassifier
Jacard score: 52.412038927453374
----
Clf:  LogisticRegression
Jacard score: 51.1014699877501
----
Clf:  LinearSVC
Jacard score: 62.42105621342044
----


In [35]:
lr_updated = LinearSVC(C=1.5, penalty = 'l1', dual=False)
for classifier in [lr_updated]:
  clf = OneVsRestClassifier(classifier)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print_score(y_pred, classifier)

Clf:  LinearSVC
Jacard score: 63.876071866067775
----


# Model prediction

In [48]:
x = [ 'how to write ml code in python and SQL i have data but do not know how to do it']
xt = tfidf.transform(x)
clf.predict(xt)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

In [49]:
multilabel.inverse_transform(clf.predict(xt))

[('python', 'sql')]